##Assignment 4 (part 1)(CPSC 436N): Count-based Sequence labeling  with Viterbi Algorithm

In Part1 of this assignment, we are going to use the the text file called __cmpt-hw2-3.txt__ to build a Hidden Markov Model to predict the part of speech tags for the words in a given sentence using the *Viterbi Algorithm* (see textbook sec 8.4.5). In class we learned about the *Forward Algorithm* and Viterbi is a slight variation of that.

## 1. Loading the dataset (cmpt-hw2-3.txt)

In order to ease later steps, we can load and save the cmpt-hw2-3.txt dataset (i.e., our corpus annotated with POSs) in a format different from its origianl one. In the cmpt-hw2-3.txt each word is followed by an underscore and a tag that represents the word’s correct part of speech in that context. For instance:

<br>

*(1) There_EX are_VBP also_RB plant_NN and_CC gift_NN shops_NNS ._.*

*(2) Tosco_NNP said_VBD it_PRP expects_VBZ BP_NNP to_TO shut_VB the_DT plant_NN ._.*

<br>

We will instead reformat and load the same info as (token, pos_tag) lists. For example:

<br>

[
  [(there, EX), (are, VBP), (also, RB), (plant, NN), (and, CC), (gift, NN), (shop, NNS), (., .)],

  [(tosco, NNP), (said, VBD), (it, PRP), (expects, VBZ), (BP, NNP), (to, To), (shut, VB), (the, DT), (plant, NN), (., .)]
]

<br>

**Please note that we convert each token into its lower case.**

Also in this step, we  obtain the set of states for the HHM (the POSs)  and set of observation (the tokens).


In [1]:
data_path = './cmpt-hw2-3.txt'

dataset = []; states = []; observations = []; #initialize dataset, state and observation list.
'''
https://medium.com/data-science-in-your-pocket/pos-tagging-using-hidden-markov-models-hmm-viterbi-algorithm-in-nlp-mathematics-explained-d43ca89347c4
- Tags are hidden states
- words are observations
- every word has a state (tag) which is "hidden"
- Given an input as HMM (Transition Matrix, Emission Matrix) and a sequence of observations O = o1, o2, …, oT 
(Words in sentences of a corpus), find the most probable sequence of states Q = q1q2q3 …qT (POS Tags in our case)

https://www.mygreatlearning.com/blog/pos-tagging/
Now, what is the probability that the word Ted is a noun, will is a model, spot is a verb and Will is a noun. These sets of 
probabilities are Emission probabilities and should be high for our tagging to be likely.

'''
for line in open(data_path): #load data from the text file.
  text_pos_list = [];
  l = line.strip().split(' ') #split each line in the file by space.
  for w in l:
    w = w.split('_') #each token and its pos tag are connected by an underscore, here we split it by underscore.
    states.append(w[1]) #add pos tag to the initial state list.
    observations.append(w[0].lower()) #lowercase the token and add it to the initial observation list.
    text_pos_list.append((w[0].lower(), w[1])) #add the (token, pos) pair into the list for the sentence.
  dataset.append(text_pos_list) #add the processed sentence list into the dataset list.

## 2. Further processing the dataset, state and observation list

In this step, we need to consider two issue:

**Q1(a)** What is the problem with the state lists we just obtained? report the problem and fix it in the code below.


**Q1(b)** The first pass over the dataset generates a fixed list of vocabulary tokens. What would happen if the input sentence we want to tag contained unknown tokens, i.e which are not covered in the vocabulary list? 
The code below is implementing a possible way to adderess this problem. Please add comments to the lines with "COMMENT NEEDED". And describe in your words the implemented solution



In [2]:
#SOLUTION Q1(a)

#*************

unk_list = []; #COMMENT NEEDED Q1(b). An array to keep track of all unknown word/tag combinations

for o in tuple(set(observations)):
  if observations.count(o) < 2:
     unk_list.append(o) # COMMENT NEEDED Q1(b) a word is unknown if it was only seen once 

for i in range(len(observations)):
  if observations[i] in unk_list:
     observations[i] = '_unk_' #COMMENT NEEDED Q1(b) if a word was labelled as unknown, change it to a homogenoues form (_unk_)

observations = tuple(set(observations)) #COMMENT NEEDED Q1(b) remove duplicate words in observations i.e make it a set
states = tuple(set(states))

for text_pos_i in range(len(dataset)): #COMMENT NEEDED Q1(b) traverse all sentences and change any unknown words to _unk_
  for w_i in range(len(dataset[text_pos_i])):
    if dataset[text_pos_i][w_i][0] in unk_list:
       dataset[text_pos_i][w_i] = ('_unk_', dataset[text_pos_i][w_i][1]) 


## 3. Computing the Emission Probabilities

The emission probability matrix can be formed as a dictionary, **Q2:** please try to understand the structure of this dictionary and complete the following code to obtain the emission probability matrix with a final normalization+smoothing step



In [23]:
from collections import defaultdict
def normalize_and_smoothing(d, all_elements):
   for e in all_elements:
     if e not in d.keys():
       d[e] = 0
   raw = sum(d.values())+len(d)
   factor = float(1)/raw
   output = {key:(value+1)*factor for key,value in d.items()}
   return output

emission_probability = {} #initialize the emission probability matrix.
for sent in dataset: #traverse the dataset to obtain the emission probability matrix.
  for i in range(len(sent)):
    if sent[i][1] not in emission_probability.keys(): #if the state (pos) is not in the key list of emission probability dict yet, intiailize a subdict for it and add the corresponding observation (token).
       emission_probability[sent[i][1]] = {}
       emission_probability[sent[i][1]][sent[i][0]] = 1
    else: #if the state (pos) is in the key list of emission probability dict, update the frequency of the corresponding observation (token).
      if sent[i][0] not in emission_probability[sent[i][1]].keys():
         emission_probability[sent[i][1]][sent[i][0]] = 1
      else:
         emission_probability[sent[i][1]][sent[i][0]] += 1
            
#****** SOLUTION Q2 *******
# states_count = defaultdict(int)

    
      
# for state, word_count_dict in emission_probability.items():
#     states_count[state] += sum(word_count_dict.values())


# for state, word_count_dict in emission_probability.items():
#     for word, count in word_count_dict.items():
#         emission_probability[state][word] = emission_probability[state][word]/states_count[state]
        
# for state, word_count_dict in emission_probability.items():
#     emission_probability[state] = normalize_and_smoothing(word_count_dict, observations)
        
# # print(emission_probability)
for state, word_count_dict in emission_probability.items():
    emission_probability[state] = normalize_and_smoothing(word_count_dict, observations)
        
# for state, word_count_dict in emission_probability.items():
#     print(sorted(word_count_dict.values()))
#     print("\n")
# print(emission_probability)
            
'''
Go through every tag
    Count the total number of times they're seen
Go through every tag
    Divide the number of times they're seen with a word by the total number of times they're seen

'''

#*************

"\nGo through every tag\n    Count the total number of times they're seen\nGo through every tag\n    Divide the number of times they're seen with a word by the total number of times they're seen\n\n"

## 4. Computing the Start and Transition Probabilities

The Transition and Strat probability matrices can be similarly formed as dictionaries,** Q2 cont'** please implement the normalization function also for the transition probabilitie

In [24]:
transition_probability = {} #initialize the transition probability matrix.

for sent in dataset: #traverse the dataset to obtain the transition probability matrix.
  for i in range(len(sent)-1):
    if sent[i][1] not in transition_probability.keys(): #if the current state (pos) is not in the key list of transition probability dict yet, intiailize a subdict for it and add its subsequent state.
       transition_probability[sent[i][1]] = {}
       transition_probability[sent[i][1]][sent[i+1][1]] = 1
    else: #if the state (pos) is in the key list of transition probability dict, update the frequency of its subsequent state.
      if sent[i+1][1] not in transition_probability[sent[i][1]].keys():
         transition_probability[sent[i][1]][sent[i+1][1]] = 1
      else:
         transition_probability[sent[i][1]][sent[i+1][1]] += 1

#********* SOLUTION Q2 *******
# next_states_count = defaultdict(int)
# for state, next_state_count_dict in transition_probability.items():
#     next_states_count[state] += sum(next_state_count_dict.values())

# for state, next_state_count_dict in transition_probability.items():
#     for next_state, count in next_state_count_dict.items():
#         transition_probability[state][next_state] = transition_probability[state][next_state]/next_states_count[state]
        
# for state, next_state_count_dict in transition_probability.items():
#     transition_probability[state][next_state_count_dict] = normalize_and_smoothing(next_state_count_dict, states)
#     emission_probability[state][word] = normalize_and_smoothing(word_count_dict, observations)
    
for state, next_state_count_dict in transition_probability.items():
    transition_probability[state] = normalize_and_smoothing(next_state_count_dict, states)
        
# for state, word_count_dict in transition_probability.items():
#     print(sorted(word_count_dict.values()))
#     print("\n")

#*******************************


start_probability = {} #initialize the transition probability matrix.
for sent in dataset: #construct the start probability dictionary which contains the frequency of each state appearing at the start position of a sentence.
  if sent[0][1] not in start_probability.keys():
     start_probability[sent[0][1]] = 1
  else:
     start_probability[sent[0][1]] += 1
start_probability = normalize_and_smoothing(start_probability, states) #normalize the start probability dict.

## 5. Implementing the Viterbi Algorithm 

The code below is implementing the Viterbi algorithm. <br>
**Q3**: Please add comments to the lines with "COMMENT NEEDED". 

In [25]:
def Viterbi(input_observations, states, start_probability, transition_probability, emission_probability):
    path = {s:[] for s in states} #this dictionary saves the shortest path for each state.
    curr_probability = {}
    for s in states:
        curr_probability[s] = start_probability[s]*emission_probability[s][input_observations[0]] #"COMMENT NEEDED" (Q3)
    for i in range(1, len(input_observations)):
        last_probability = curr_probability #update the current state probabilities to the ones obtained in the last step.
        curr_probability = {} #initialize a new dict to save the state probabilities for this step. 
        for curr_state in states: #for each possible state in this step, compute all possible paths and their probabilities towards this state.
            if input_observations[i] not in emission_probability[curr_state]:
                possible_paths = [(last_probability[last_state]*transition_probability[last_state][curr_state]*min(emission_probability[curr_state].values()), last_state) for last_state in states]
            else:
                possible_paths = [(last_probability[last_state]*transition_probability[last_state][curr_state]*emission_probability[curr_state][input_observations[i]], last_state) for last_state in states]
            max_probability, last_state = max(possible_paths) #"COMMENT NEEDED" (Q3)
            curr_probability[curr_state] = max_probability #update the current state probabilities.
            path[curr_state].append(last_state) #record the optimal path towards each possible state in this step.
    for s in states:
        path[s].append(s) #complete the path with each state as the end.

   #"COMMENT NEEDED" (Q3)
    sorted_probability = sorted(curr_probability.items(), key=lambda x: x[1], reverse=True)
    the_last_state = sorted_probability[0][0]
    optimal_path = path[the_last_state]

    return optimal_path

## 6. Applying the the Viterbi Algorithm 

**Q4:** Apply the Viterbi Algorithm to the following sentences (remember to deal with unknown words): <br>
(a) "there are also many plants in the square." <br>
(b) "we will move the table into cs123." <br>

Please report the tagging results.

In [26]:
# SOLUTION Q4(a)
input_a = ["there", "are", "also", "many", "plants", "in", "the", "square", "."]
sol_a = Viterbi(input_a, states, start_probability, transition_probability, emission_probability)
print(sol_a)


['RB', 'VBP', 'RB', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.']


In [28]:
# SOLUTION Q4(b)
input_b = ["we", "will", "move", "the", "tables", "into", "cs123", "."]
sol_b = Viterbi(input_b, states, start_probability, transition_probability, emission_probability)
print(sol_b)


['PRP', 'MD', 'VB', 'DT', 'NN', 'NN', 'NNP', '.']
